In [ ]:
!pip install openai
!pip install langchain
!pip install google-search-results

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.7/993.7 kB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel fo

In [ ]:
from langchain import OpenAI

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

from langchain.utilities import SerpAPIWrapper

In [ ]:
## 사용자키 설정
import os

os.environ["OPENAI_API_KEY"] = "sk-##"
os.environ["SERPAPI_API_KEY"] = "##"

In [ ]:
# 사용할 언어 모델 설정
llm = OpenAI(temperature=0)

In [ ]:
# 사용할 도구 설정
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [ ]:
# 에이전트 선언
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
# 에이전트 실행
agent.run("대한민국의 대통령을 2023년 기준으로 대한민국의 대통령이 누군지 찾아라. 해당 대통령의 현재 (2023년) 나이를 계산해라.")



> Entering new AgentExecutor chain...
 대한민국의 대통령을 찾고 그 나이를 계산해야 한다.
Action: Search
Action Input: 대한민국 대통령 2023
Observation: Yoon Suk Yeol is a South Korean politician who has been serving as the 13th and current president of South Korea since 2022. Prior to his presidency, he served as the prosecutor general of South Korea between 2019 and 2021. Born in Seoul, Yoon attended Seoul National University.
Thought:

 Yoon Suk Yeol의 생년월일을 찾아야 한다.
Action: Search
Action Input: Yoon Suk Yeol 생년월일
Observation: December 18, 1960
Thought: Yoon Suk Yeol의 나이를 계산해야 한다.
Action: Calculator
Action Input: 2023 - 1960
Observation: Answer: 63
Thought: I now know the final answer
Final Answer: Yoon Suk Yeol은 2023년 기준으로 63세이다.

> Finished chain.


'Yoon Suk Yeol은 2023년 기준으로 63세이다.'

In [ ]:
print(agent.tools[0])

name='Search' description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.' args_schema=None return_direct=False verbose=False callbacks=None callback_manager=None handle_tool_error=False func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='43d7ab687676d6de83bfbd031578c8c4496e39f73012094ae129bfd8b6887e95', aiosession=None)> coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='43d7ab687676d6de83bfbd031578c8c4496e39f73012094ae129bfd8b6887e95', aiosession=None)>


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain import LLMMathChain


In [ ]:
search = SerpAPIWrapper()
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]


In [ ]:
# 사용할 언어 모델 선택 
model = ChatOpenAI(temperature=0)
# 플래너 설정
planner = load_chat_planner(model)
# 실행자 설정
executor = load_agent_executor(model, tools, verbose=True)
# 에이전트 선언
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

# 에이전트 실행
agent.run("대한민국의 대통령을 2023년 기준으로 대한민국의 대통령이 누군지 찾아라. 대통령의 생년월일과 2023년를 비교하여 현재 나이를 계산해라.")




> Entering new PlanAndExecute chain...
steps=[Step(value='대한민국 대통령 목록을 찾아서 2023년 기준으로 누가 대통령인지 확인한다.'), Step(value='대통령의 생년월일을 찾아서 현재 나이를 계산한다.'), Step(value='계산된 현재 나이를 출력한다.\n')]

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Search",
  "action_input": "List of Presidents of South Korea as of 2023"
}
```


Observation: Yoon Suk Yeol is a South Korean politician who has been serving as the 13th and current president of South Korea since 2022. Prior to his presidency, he served as the prosecutor general of South Korea between 2019 and 2021. Born in Seoul, Yoon attended Seoul National University.
Thought:

The human is looking for the current president of South Korea as of 2023. I have found that Yoon Suk Yeol is the current president of South Korea as of 2023. 

Action:
```
{
  "action": "Final Answer",
  "action_input": "The current president of South Korea as of 2023 is Yoon Suk Yeol."
}
```



> Finished chain.
*****

Step: 대한민국 대통령 목록을 찾아서 2023년 기준으로 누가 대통령인지 확인한다.

Response: The current president of South Korea as of 2023 is Yoon Suk Yeol.

> Entering new AgentExecutor chain...


Action:
```
{
  "action": "Search",
  "action_input": "What is the birthdate of Yoon Suk Yeol, the current president of South Korea?"
}
```

Observation: Yoon Suk Yeol (Korean: 윤석열; born 18 December 1960) is a South Korean politician who has been serving as the 13th and current president of South Korea ...
Thought:To calculate the current age of Yoon Suk Yeol, I will need to get the current date and subtract his birthdate from it. 

Action:
```
{
  "action": "Calculator",
  "action_input": "2023-12-18 - 1960-12-18"
}
```



> Entering new LLMMathChain chain...
2023-12-18 - 1960-12-18

```text
2023-1960
```
...numexpr.evaluate("2023-1960")...

Answer: 63
> Finished chain.

Observation: Answer: 63
Thought:

To answer the human's question, I will provide the current age of Yoon Suk Yeol, which is 63.

Action:
```
{
  "action": "Final Answer",
  "action_input": "Yoon Suk Yeol's current age is 63."
}
```



> Finished chain.
*****

Step: 대통령의 생년월일을 찾아서 현재 나이를 계산한다.

Response: Yoon Suk Yeol's current age is 63.

> Entering new AgentExecutor chain...


Action:
```
{
  "action": "Final Answer",
  "action_input": "Yoon Suk Yeol's current age is 63."
}
```


> Finished chain.
*****

Step: 계산된 현재 나이를 출력한다.


Response: Yoon Suk Yeol's current age is 63.
> Finished chain.


"Yoon Suk Yeol's current age is 63."

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 불러올 데이터베이스 선택
db = SQLDatabase.from_uri("sqlite:////content/drive/My Drive/Colab Notebooks/sqlite-sakila.db")
# 툴킷 선언
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
# 에이전트 선언
agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True
)
# 에이전트 실행
agent_executor.run("고객 테이블에 대해서 설명해봐.")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: customer, actor, inventory, film_actor, rental, film_text, address, film, category, store, staff, country, language, city, film_category, payment
Thought: I should query the schema of the customer table
Action: schema_sql_db
Action Input: "customer"
Observation: 
CREATE TABLE customer (
	customer_id INTEGER NOT NULL, 
	store_id INTEGER NOT NULL, 
	first_name VARCHAR(45) NOT NULL, 
	last_name VARCHAR(45) NOT NULL, 
	email VARCHAR(50) DEFAULT NULL, 
	address_id INTEGER NOT NULL, 
	active CHAR(1) DEFAULT 'Y' NOT NULL, 
	create_date TIMESTAMP NOT NULL, 
	last_update TIMESTAMP NOT NULL, 
	PRIMARY KEY (customer_id), 
	CONSTRAINT fk_customer_store FOREIGN KEY(store_id) REFERENCES store (store_id) ON UPDATE CASCADE, 
	CONSTRAINT fk_customer_address FOREIGN KEY(address_id) REFERENCES address (address_id) ON UPDATE CASCADE
)

/*
3 rows from customer table:
customer_id	store_id	first_name	last_name	em

'customer 테이블은 customer_id, store_id, first_name, last_name, email, address_id, active, create_date, last_update 의 열을 가지고 있으며, 각 열은 각각 INTEGER, VARCHAR, CHAR, TIMESTAMP 타입을 가지고 있습니다.'

In [ ]:
agent_executor.run("고객이 가장 많이 빌린 영화를 찾아줘.")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: customer, actor, inventory, film_actor, rental, film_text, address, film, category, store, staff, country, language, city, film_category, payment
Thought: I should query the schema of the film, rental, and customer tables.
Action: schema_sql_db
Action Input: "film, rental, customer"
Observation: 
CREATE TABLE film (
	film_id INTEGER NOT NULL, 
	title VARCHAR(255) NOT NULL, 
	description TEXT DEFAULT NULL, 
	release_year VARCHAR(4) DEFAULT NULL, 
	language_id SMALLINT NOT NULL, 
	original_language_id SMALLINT DEFAULT NULL, 
	rental_duration SMALLINT DEFAULT 3 NOT NULL, 
	rental_rate DECIMAL(4, 2) DEFAULT 4.99 NOT NULL, 
	length SMALLINT DEFAULT NULL, 
	replacement_cost DECIMAL(5, 2) DEFAULT 19.99 NOT NULL, 
	rating VARCHAR(10) DEFAULT 'G', 
	special_features VARCHAR(100) DEFAULT NULL, 
	last_update TIMESTAMP NOT NULL, 
	PRIMARY KEY (film_id), 
	CONSTRAINT fk_film_language FOREIGN KEY(languag

"가장 많이 빌린 영화는 'BUCKET BROTHERHOOD'입니다."